# Projeto 1 - Ciência dos Dados

Nome: Alison Araujo

Nome: Gabrielly Carneiro

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [143]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('word_tokenize')
stopwordsdic = stopwords.words('portuguese')

from spacy import load
nlp = load('pt_core_news_sm')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [144]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Projeto 1\C.DadosP1


Carregando a base de dados com os tweets classificados manualmente:

In [145]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"Para quem gosta de poemas simples, esse é o li...",N
1,Quando vi o lançamento pensei que finalmente p...,D
2,"É incrível como esses escritores e ""intelectua...",N
3,"Se desse pra devolver eu devolvia, nao é por e...",N
4,"Futilidade, inutilidade, desperdício de papel,...",N


In [146]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Acionável/Direcionável/Não Acionável
0,"O livro é prolixo, redundante, doentio. Sou su...",N
1,"Bom livro e história envolvente. Porém, o leit...",D
2,"Fala tudo que todos já sabem, sem falar que nã...",N
3,Ganhei na compra do Kindle. Não é estilo de li...,N
4,"Custa crer que um livro tão medíocre, embora m...",N


___
## Classificador automático


Na primeira etapa, de classificação manual, consideramos três targets para os reviews: Acionável, Direcionável e Não Acionável. 

- Acionável: para ser considerado "acionável" ("A") o review deve ser passível de alguma ação pela Amazon, ou seja, o review deve ser sobre entrega, estado do produto, contato com o suporte, etc. 
- Direcionável: para o target "direcionável" ("D") foram considerados comentários relativos à editora, como qualidade do material do livro, preço do livro e do e-book, tradução e edição. 
- Não Acionáveis: por fim, os não acionáveis ("NA") eram comentários relativos ao autor, ao apreço pelo conteúdo do livro, ou comentários irrelavantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Funções úteis:

In [147]:
#Criar uma função que transforma as frases da planilha em um texto só 
    #(Será útil para criar o dicionário com as palavras)
def transforma_em_string(coluna):
    texto = ''
    for linha in coluna:
        texto += linha + ' '
    return texto    

In [148]:
#Criar uma função que limpa todas as pontuações
def cleanup(text):
    punctuation = r'[´"\'!-.:?;$,/~^_=+*&¨%$#@|\{}()[\]]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [149]:
#Cria uma função que limpa os espaços duplicados
def limpa_espaco(text):
    punctuation = r'[\n]'  # Adicione os caracteres desejados aqui
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [150]:
#Criando uma função para remover emoji
def remove_emoji(text):
    text_without_emojis = unidecode(text)
    return text_without_emojis

In [151]:
#Criando a função de stopwords
def stopwords(texto):
    palavras = word_tokenize(texto, language='portuguese') # Tokenize é analisar palavras individualmente, basicamente
    palavras_sem_stopword = []
    for palavra in palavras:
        if palavra not in stopwordsdic:
            palavras_sem_stopword.append(palavra)
    # Reúna as palavras sem stopwords em uma string novamente
    texto_sem_stopword = ' '.join(palavras_sem_stopword)
    return texto_sem_stopword

In [152]:
#Criando a função de lematização
def lemmat(texto):
    doc = nlp(texto)
    lemmat_radicais = []
    for radicais in doc:
        lemmat_radicais.append(radicais.lemma_)
    texto_lemmat = ' '.join(lemmat_radicais)    
    return texto_lemmat

In [153]:
# Cria uma função que reúna as funções de limpeza
def limpa_tudo(mensagem):
    #Aplicando a função de limpeza de pontuação
    texto = cleanup(mensagem)
    #Deixando tudo em letra minúscula
    texto = texto.lower()
    #Removendo emoji
    texto = remove_emoji(texto)
    #Aplicando a função de limpeza de espaço
    texto = limpa_espaco(texto)
    #Removendo stopwords
    texto = stopwords(texto)
    #Realiza lemmatização
    # texto = lemmat(texto)          # removi pois demora muito para classificar 
    return texto


In [154]:
#Cria uma função que limpa as linhas da planilha e adiciona as mensagens limpas à planilha
def mensagem_limpa(planilha):   #recebe a planilha e retorna com a coluna de mensagem limpa
    planilha['Mensagem Limpa'] = [limpa_tudo(x) for x in list(planilha['Mensagem'])]
    return planilha


In [155]:
#Cria vocabulário de tudo
def cria_vocabulario(coluna_da_planilha):                       #recebe uma coluna da planilha
    lista_palavras = transforma_em_string(coluna_da_planilha)
    lista_palavras = limpa_tudo(lista_palavras)
    lista_palavras = lista_palavras.split()
    return lista_palavras     #devolve uma lista com as palavras separadas~


In [156]:
#Cria função que retorna uma lista sem as palavras repetidas
def remove_repeticao(lista):
    dic = set(lista)
    vocabulario = list(dic)
    return vocabulario

In [157]:
# Cria uma função que guarda as palavras em pd.Series
def cria_pdseries(lista):
    tabela = pd.Series(lista)
    return tabela

#Cria uma função que retorna a frequência absoluta de cada palavra no texto
def freq_abs(tabela):
    absoluta = tabela.value_counts()
    return absoluta

#Cria uma função que retorna a frequência relativa de cada palavra no texto
def freq_rel(tabela):
    relativa = tabela.value_counts(True)
    return relativa

In [158]:
## Cria uma função que recebe a planilha e o target e retorna o total de palavras e as tabelas de frequencia absoluta e relativa, nessa ordem
def divisao_categorias(planilha, target):
    #Etapa de divisão de categorias
    planilha= mensagem_limpa(planilha)
    
    if target == "N":
        filtro_target = planilha.loc[planilha['Acionável/Direcionável/Não Acionável'] =='N']
        vocab_target = cria_vocabulario(filtro_target["Mensagem Limpa"])
        df_vocab_target = cria_pdseries(vocab_target)
        freq_rel_target = freq_rel(df_vocab_target)
        freq_abs_target = freq_abs(df_vocab_target)
        total_target = freq_abs_target.sum()
        #criar vocabulario limpo
        vocab_target = remove_repeticao(vocab_target)
    elif target == "D":
        filtro_target = planilha.loc[planilha['Acionável/Direcionável/Não Acionável'] =='D']
        vocab_target = cria_vocabulario(filtro_target["Mensagem Limpa"])
        df_vocab_target = cria_pdseries(vocab_target)
        freq_rel_target = freq_rel(df_vocab_target)
        freq_abs_target = freq_abs(df_vocab_target)
        total_target = freq_abs_target.sum()
        #criar vocabulario limpo
        vocab_target = remove_repeticao(vocab_target)
    elif target == "A":
        filtro_target = planilha.loc[planilha['Acionável/Direcionável/Não Acionável'] =='A']
        vocab_target = cria_vocabulario(filtro_target["Mensagem Limpa"])
        df_vocab_target = cria_pdseries(vocab_target)
        freq_rel_target = freq_rel(df_vocab_target)
        freq_abs_target = freq_abs(df_vocab_target)
        total_target = freq_abs_target.sum()
        #criar vocabulario limpo
        vocab_target = remove_repeticao(vocab_target)
    return total_target, freq_abs_target, freq_rel_target, vocab_target

In [159]:
## Função que recebe a planilha e retorna:
# o total de palavras com repetição [0] (int)
# a tabela com a frequência absoluta de cada palavra na planilha [1] (Series)
# a tabela com a frequência relativa de cada palavra na planilha [2] (Series)
# a lista com todas as palavras da planilha sem repetição [3] (list)
def divisao_planilhas(planilha):

    planilha= mensagem_limpa(planilha)                      #cria coluna "Mensagem Limpa"
    planilha = planilha["Mensagem Limpa"]                   #separa só a coluna "Mensagem Limpa"
    
    vocab_planilha = cria_vocabulario(planilha)             #lista de palavras na coluna "Mensagem Limpa"
    df_vocab_planilha = cria_pdseries(vocab_planilha)       #coloca essa lista em um df

    freq_abs_planilha = freq_abs(df_vocab_planilha)         #calcula a frequência absoluta de cada palavra
    freq_rel_planilha = freq_rel(df_vocab_planilha)         #calcula a frequência relativa de cada palavra

    total_planilha = freq_abs_planilha.sum()                #calcula o total de palavras com repetição

    vocab_planilha = remove_repeticao(vocab_planilha)

    return total_planilha, freq_abs_planilha, freq_rel_planilha, vocab_planilha

In [160]:
#Função de Suavização de LaPlace
# A função recebe a palavra, o target e a planilha
def suavizacao(palavra, target, planilha):
    #extraindo dados da planilha
    dados_target = divisao_categorias(planilha, target)
    qtdd_palavras = dados_target[0]                  #quantidade de palavras no target informado
    qtdd_palavras_sem_repeticao = len(dados_target[3]) #quantidade de palavras sem repetição no target informado
    if palavra in dados_target[3]:
        freq_abs_palavra = dados_target[1][palavra]      #frequencia absoluta da palavra nesse target
        # freq_rel_palavra = dados_target[2][palavra]      #frequencia relativa da palavra nesse target
        Prob_palavra_dado_target = (freq_abs_palavra + 1)/(qtdd_palavras + qtdd_palavras_sem_repeticao)
    else:
        freq_abs_palavra = 0
        Prob_palavra_dado_target = (freq_abs_palavra + 1)/(qtdd_palavras + qtdd_palavras_sem_repeticao)
        
    return Prob_palavra_dado_target

# exemplo de como usar
# print(suavizacao("absurdo", "N", train))

In [169]:
# Calculando probabilidade da frase dado target
# recebe uma frase qualquer (não precisa estar limpa)
def classifica_por_target(frase, target, planilha):
    prob_frase_dado_classe = 1
    frase = limpa_tudo(frase)
    frase =  frase.split()
    for palavra in frase:
        prob_palavra_dado_classe = suavizacao(palavra, target, planilha)
        prob_frase_dado_classe *= prob_palavra_dado_classe
    return prob_frase_dado_classe


In [ ]:

def classificador(frase, planilha):
    
    #Extraindo as informações
    total_train_N = divisao_categorias(train, "N")[0]
    total_train_D = divisao_categorias(train, "D")[0]
    total_train_A = divisao_categorias(train, "A")[0]
    total_train = divisao_planilhas(train)[0]
    
    #Cálculo das probabilidades
    P_frase_dado_A = classifica_por_target(frase, "A", planilha)
    P_frase_dado_D = classifica_por_target(frase, "D", planilha)
    P_frase_dado_N = classifica_por_target(frase, "N", planilha)
    P_train_N = total_train_N/total_train         #probabilidade de estar na categoria N
    P_train_D = total_train_D/total_train         #probabilidade de estar na categoria D
    P_train_A = total_train_A/total_train         #probabilidade de estar na categoria A
    P_A_dado_frase = P_frase_dado_A*P_train_A 
    P_N_dado_frase = P_frase_dado_N*P_train_N
    P_D_dado_frase = P_frase_dado_D*P_train_D
        
    #Classificação
    if P_A_dado_frase > P_D_dado_frase and P_A_dado_frase > P_N_dado_frase:
        return "A"
    elif P_D_dado_frase > P_N_dado_frase and P_D_dado_frase > P_A_dado_frase:
        return "D"
    elif P_N_dado_frase > P_D_dado_frase and P_N_dado_frase > P_A_dado_frase:
        return "N"
    else:
        return "Houve um impasse"

frase = "O livro é prolixo, redundante, doentio"

print(classificador(frase, train))
    

Calculando as informações da base de treino

In [168]:
####-  DIVISÃO CATEGORIAS TREINO  -####

#--- categoria Não Acionável
# total_train_N = divisao_categorias(train, "N")[0]         #Quantidade total de palavras (incluindo as repetições)
# freq_abs_train_N = divisao_categorias(train, "N")[1]      #Tabela com as frequencias absolutas das palavras
# freq_rel_train_N = divisao_categorias(train, "N")[2]      #Tabela com as frequencias relativas das palavras
# vocab_train_N = divisao_categorias(train, "N")[3]         #Vocabulario de todas as palavras dessa classificação (SEM as repetições)

# #--- categoria Direcionável
# total_train_D = divisao_categorias(train, "D")[0]
# freq_abs_train_D = divisao_categorias(train, "D")[1]
# freq_rel_train_D = divisao_categorias(train, "D")[2]
# vocab_train_D = divisao_categorias(train, "D")[3]              

# #--- categoria Acionável
# total_train_A = divisao_categorias(train, "A")[0]
# freq_abs_train_A = divisao_categorias(train, "A")[1]
# freq_rel_train_A = divisao_categorias(train, "A")[2]
# vocab_train_A = divisao_categorias(train, "A")[3]       

# #--- Todas as palavras
# total_train = divisao_planilhas(train)[0]
# freq_abs_train = divisao_planilhas(train)[1]
# freq_rel_train = divisao_planilhas(train)[2]
# vocab_train = divisao_planilhas(train)[3]

# #### Guardando as palavras em um pd.Series
# todas_palavras_train = cria_pdseries(vocab_train)
# palavras_train_A = cria_pdseries(vocab_train_A)
# palavras_train_N = cria_pdseries(vocab_train_N)
# palavras_train_D = cria_pdseries(vocab_train_D)

####- PROBABILIDADES DAS CATEGORIAS TREINO -####
# prob_trein_N = total_train_N/total_train         #probabilidade de estar na categoria N
# prob_trein_D = total_train_D/total_train         #probabilidade de estar na categoria D
# prob_trein_A = total_train_A/total_train         #probabilidade de estar na categoria A

# prob_trein_N + prob_trein_A + prob_trein_D       #conferindo


Palavras da base de teste

In [ ]:
# ####-  DIVISÃO CATEGORIAS TESTE  -####

# #--- categoria Não Acionável
# total_test_N = divisao_categorias(test, "N")[0]         #Quantidade total de palavras (incluindo as repetições)
# freq_abs_test_N = divisao_categorias(test, "N")[1]      #Tabela com as frequencias absolutas
# freq_rel_test_N = divisao_categorias(test, "N")[2]      #Tabela com as frequencias relativas
# vocab_test_N = divisao_categorias(test, "N")[3]         #vocabulario com as palavras dessa categoria (SEM as repetições)

# #--- categoria Direcionável
# total_test_D = divisao_categorias(test, "D")[0]         
# freq_abs_test_D = divisao_categorias(test, "D")[1]
# freq_rel_test_D = divisao_categorias(test, "D")[2]
# vocab_test_D = divisao_categorias(test, "D")[3]  

# #--- categoria Acionável
# total_test_A = divisao_categorias(test, "A")[0]
# freq_abs_test_A = divisao_categorias(test, "A")[1]
# freq_rel_test_A = divisao_categorias(test, "A")[2]
# vocab_test_A = divisao_categorias(test, "A")[3]  

# #--- Todas as palavras
# total_test = divisao_planilhas(test)[0]
# freq_abs_test = divisao_planilhas(test)[1]
# freq_rel_test = divisao_planilhas(test)[2]
# vocab_test = divisao_planilhas(test)[3]

# #### Guardando as palavras em um pd.Series
# todas_palavras_test = cria_pdseries(vocab_test)         #cria um dataframe com todas as palavras da planilha de teste
# palavras_test_A = cria_pdseries(vocab_test_A)           #cria um df com todas as palavras do target acionavel
# palavras_test_N = cria_pdseries(vocab_test_N)           #cria um df com todas as palavras do target nao acionavel
# palavras_test_D = cria_pdseries(vocab_test_D)           #cria um df com todas as palvras do target direcionavel
# """EXPLICAÇÕES
# vocab_df [lista] = palavras já limpas

# df_vocab_df [tabela] = transforma o vocab_df em tabela

# freq_rel_df [tabela] = frequencia relativa de cada palavra
# freq_abs_df [tabela] = 1
# total_df [int] = quantidade total de palavras
# """
# ####- PROBABILIDADES DAS CATEGORIAS TESTE -####
# prob_test_N = total_test_N/total_test    #probabilidade de estar na categoria N
# prob_test_D = total_test_D/total_test    #probabilidade de estar na categoria D
# prob_test_A = total_test_A/total_test    #probabilidade de estar na categoria A

# prob_test_A + prob_test_D + prob_test_N  #conferindo

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [175]:
## Fazer meu classificador classificar todas as frases da planilha de teste

def classifica_planilha(planilha):
    planilha= mensagem_limpa(planilha)                      #cria coluna "Mensagem Limpa"
    avaliacoes_limpas = planilha["Mensagem Limpa"]          #separa só a coluna "Mensagem Limpa"
    df_avaliacoes_limpas = cria_pdseries(avaliacoes_limpas)
    
    for frase in df_avaliacoes_limpas:
        planilha['Classificador Automático'] = classificador(frase, planilha)
    
    return planilha
        
# classifica_planilha(train)
    

___
### Concluindo

In [ ]:
#a probabilidade de todas as palavras estarem contidas num texto não é a mesma? 

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Dica: apresentar um grafico com testes mostrando quais limpezas melhoraram a acurácia do nosso classificador (lemmatization, stopwords, etc)

REtirar algumas palavras para ver se melhora a qualidade do classificador (exemplo palavra não)